In [1]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_community.document_loaders import WebBaseLoader

In [3]:
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
data = loader.load()
data

[Document(page_content="\n\n\n\n\nLangSmith User Guide | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly un

In [5]:
%pip install langchain_objectbox

  Obtaining dependency information for langchain_objectbox from https://files.pythonhosted.org/packages/66/42/f121f75f28556021fa0e654da3052ce16c7b5cce166f9e6d73096a08653f/langchain_objectbox-0.1.0a3-py3-none-any.whl.metadata
  Obtaining dependency information for objectbox==0.7.0a10 from https://files.pythonhosted.org/packages/07/29/7c16b2b7795bc15436130f2afaa84855bdd1d0c3c2830e81627458c76b6f/objectbox-0.7.0a10-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers==23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.0 MB 435.7 kB/s eta 0:00:10
   ---------------------------------------- 0.0/4.0 

In [6]:
from langchain_objectbox.vectorstores import ObjectBox ##vector Database
from langchain_openai import OpenAIEmbeddings

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(data)

In [8]:
documents

[Document(page_content='LangSmith User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.', 'language': 'en'}),
 Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform

In [9]:
from langchain_openai import OpenAIEmbeddings
vector = ObjectBox.from_documents(documents, OpenAIEmbeddings(), embedding_dimensions=768)
vector

# 3. Make a RAG pipeline

In [10]:
from langchain_openai import ChatOpenAI #this is for interacting as a chat
from langchain_core.output_parsers import StrOutputParser #responsible for displaying the output from LLM model
from langchain_core.prompts import ChatPromptTemplate # Creating your own prompt template
from langchain.chains import RetrievalQA
from langchain import hub # from the hub where i can download the my rag prompt

In [13]:
prompt_template="""
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [25]:
from langchain import PromptTemplate
llm = ChatOpenAI() 
# llm = ChatOpenAI(model="gpt-4o") ## Calling Gpt-4o
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])


In [26]:
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vector.as_retriever(),
        chain_type_kwargs={"prompt": PROMPT}
    )

In [27]:
question = "Explain what is langsmith"
result = qa_chain({"query": question })
result

{'query': 'Explain what is langsmith',
 'result': 'LangSmith is a platform for LLM application development, monitoring, and testing. It supports various workflows throughout the application development lifecycle, including prototyping, debugging, initial test sets, comparison views, playground environment, beta testing, capturing feedback, annotating traces, adding runs to datasets, production monitoring, A/B testing, automations, and threads view for multi-turn applications.'}